# Convert a tensorflow model for the generic inference runner




### Install Python Packages required

Run pip to install Tensorflow etc. into the current environment.

In [ ]:
%pip install ethos_u-vela

In [ ]:
import os
import re

def sanitize_name(name):
    """Convert filename to valid C variable name."""
    return re.sub(r'[^a-zA-Z0-9]+', '_', name)

def convert_to_c_source(input_file, output_file, bytes_per_line=12):
    """Convert binary file to C source array."""
    var_name = sanitize_name(os.path.basename(input_file))
    
    with open(input_file, 'rb') as f_in:
        content = f_in.read()
    
    with open(output_file, 'w') as f_out:
        f_out.write(f"const unsigned char {var_name}[] = {{\n  ")
        
        # Write bytes in hex format
        for i, byte in enumerate(content):
            if i > 0 and i % bytes_per_line == 0:
                f_out.write("\n  ")
            f_out.write(f"0x{byte:02x}, ")
        
        f_out.write(f"\n}};\n")
        f_out.write(f"const unsigned int {var_name}_len = {len(content)};\n")


## Convert to Ethos-U NPU model

Optionally convert the model for targets featuring the Ethos-U NPU. 

In [ ]:
import site
 
# Get the site-packages path to find vela
site_packages_path = site.getsitepackages()[0]
print(site_packages_path)
 
# Construct the installation path of vela
if os.name == 'nt':
    velapath = os.path.normpath(os.path.join(site_packages_path, 'Scripts/vela'))
else:
    velapath = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(site_packages_path))), 'bin/vela')
 
# Convert the model to a Vela model
!{velapath} models/model_int8.tflite --output-dir models_vela --verbose-operators
 
convert_to_c_source ("models_vela/model_int8_vela.tflite", "user_model.c")

In [ ]:
#Display the models_vela/model_int8_summary_Ethos_U55_High_End_Embedded.csv convertion metrics
df = pd.read_csv("models_vela/model_int8_summary_Ethos_U55_High_End_Embedded.csv")
df

## Deploy to a Project

Within this project, the model is copied into the Model layer in [`Model`] after the Jupyter Notebook executed successful. There is will be used by hello_world_test.cpp. 